In [1]:
import requests
import json
import re
import time

import pandas as pd

from parse_hh_data import download, parse
from tqdm.notebook import tqdm_notebook
from bs4 import BeautifulSoup
from random import randint

In [22]:
names = ["Go разработчик",
"Главный инженер по сопровождению",
"Эксперт направления моделирования резервов",
"Data Engineer",
"Senior Data Scientist",
"Аналитик SAS",
"Аналитик банковских рисков",
"Главный инженер по тестированию (автоматизация)",
"Ведущий DevOps инженер",
"Дежурный инженер сопровождения банковских систем",
"Дизайнер мобильных интерфейсов",
"Специалист по тестированию по направлению SAS RTDM",
"Разработчик Front-end (Middle)",
"Системный аналитик DWH",
"Аналитик системы принятия решений",
"Инженер DevOps",
"Главный разработчик Back-end Java",
"Разработчик RPA",
"Разработчик Back-end (Oracle+.net+ReactJS)",
"Разработчик Front-end (REACT)",
"Системный аналитик",
"Архитектор",
"Системный аналитик (проекты розничного блока)",
"Системный аналитик (базы данных)",
"Аналитик (web приложения)",
"Бизнес-технолог",
"Frontend разработчик",
"Руководитель разработки JAVA"]

In [26]:
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
                  "AppleWebKit/537.36 (KHTML, like Gecko)"
                  "Chrome/84.0.4147.89"
                  "Safari/537.36"}

In [53]:
result = []

for name in names:
    for page in [0, 1, 2]:
        url = "https://hh.ru/search/resume?clusters=True&area=1&order_by=relevance&logic=normal&pos=full_text&exp_period=all_time&no_magic=False&st=resumeSearch&text={}&page={}".format(
            name.replace(" ", "+"), page)
        page_res = requests.get(url, headers=headers)
        for res in re.findall("resume\/([0-9a-z]+)\?", page_res.text):
            result.append((name, page_res, res))
        print(name, page)
        time.sleep(randint(0, 10) / 10)

Go разработчик 0
Go разработчик 1
Go разработчик 2
Главный инженер по сопровождению 0
Главный инженер по сопровождению 1
Главный инженер по сопровождению 2
Эксперт направления моделирования резервов 0
Эксперт направления моделирования резервов 1
Эксперт направления моделирования резервов 2
Data Engineer 0
Data Engineer 1
Data Engineer 2
Аналитик SAS 0
Аналитик SAS 1
Аналитик SAS 2
Аналитик банковских рисков 0
Аналитик банковских рисков 1
Аналитик банковских рисков 2
Главный инженер по тестированию (автоматизация) 0
Главный инженер по тестированию (автоматизация) 1
Главный инженер по тестированию (автоматизация) 2
Ведущий DevOps инженер 0
Ведущий DevOps инженер 1
Ведущий DevOps инженер 2
Дежурный инженер сопровождения банковских систем 0
Дежурный инженер сопровождения банковских систем 1
Дежурный инженер сопровождения банковских систем 2
Дизайнер мобильных интерфейсов 0
Дизайнер мобильных интерфейсов 1
Дизайнер мобильных интерфейсов 2
Специалист по тестированию по направлению SAS RTDM 0

In [56]:
ids = pd.DataFrame(result, columns=["name", "page", "id"])[["name","id"]]

In [60]:
ids = ids[ids["id"]!="advanced"]

In [64]:
dicts = []

for user_id in tqdm_notebook(ids["id"].values):
    res = download.resume(user_id)
    dicts.append(parse.resume(res))
    time.sleep(randint(0, 4) / 10)

In [66]:
ids["data_dict"] = dicts

/home/gofat/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [68]:
ids.to_csv("parsed_data.csv", index=False)

In [81]:
total_keys = []
for i in ids["data_dict"].values:
    total_keys.extend([key for key in i.keys()])

In [109]:
def parse_experience(exp_list):
    total = ""
    for exp in exp_list:
        total += "{} {} {}\n{}\n".format(exp.get("start", ""), exp.get("end", ""), 
                                         exp.get("position", ""), exp.get("description", ""))
    return total

def parse_education(ed_list):
    total = ""
    for ed in ed_list:
        total += "{} {} {}\n".format(ed.get("year", ""), ed.get("name", ""), 
                                         ed.get("organization", ""))
    return total

def parse_spec(spec_list):
    total = ""
    for sp in spec_list:
        total += "{} {}\n".format(sp.get("name", ""), sp.get("profarea_name", ""))
    return total

def parse_lang(lang_list):
    total = ""
    for ln in lang_list:
        total += "{} {}\n".format(ln.get("name", ""), ln.get("level", ""))
    return total

In [144]:
res_text = []
for data_dict in tqdm_notebook(ids["data_dict"]):
    result = ""
    result += "{}\n".format(data_dict.get("area", ""))
    result += "{}\n".format(data_dict.get("title", ""))
    # result += "{}\n".format(data_dict.get("birth_date", ""))
    result += "{}\n".format(parse_spec(data_dict.get("specialization", [])))
    result += "{}\n".format(data_dict.get("education_level", []))
    result += "{}\n".format(parse_education(data_dict.get("education", [])))
    result += "{}\n".format(parse_experience(data_dict.get("experience", [])))
    result += "{}\n".format(parse_lang(data_dict.get("language", [])))
    result += "{}\n".format(" ".join([str(s) for s in data_dict.get("salary", {}).values()]))
    result += "{}\n".format(" ".join([s for s in data_dict.get("skill_set", [])]))
    result += "{}\n".format(data_dict.get("skills", ""))
    res_text.append(result)

In [147]:
ids["resume_text"] = res_text

/home/gofat/miniconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [148]:
ids.to_csv("parsed_data.csv", index=False)